In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [33]:
farms = pd.read_csv("../data-mining/farms.csv")
rubrics = pd.read_csv("../data-mining/rubrics.csv")
stocks = pd.read_csv("../data-mining/stocks.csv")

## Add departement column

In [34]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

farms["departement"] = farms.codePostal.apply(departement_of)

In [35]:
farms_stocks = farms.merge(stocks, left_on="name", right_on="raison_sociale")

In [36]:
farms_stocks = farms_stocks[["name", "rubric_id", "quantite", "unite", "departement", "type"]]
farms_stocks = farms_stocks[(farms_stocks.unite == "u") | (farms_stocks.unite == "Animaux-Eq")]
farms_stocks

,name,rubric_id,quantite,unite,departement,type
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq,89,PORCS
5,GAEC FERME H ET L,2101,210.0,u,50,BOVINS
6,BINOUDIERE (GAEC DE LA),2102,498.0,Animaux-Eq,50,BOVINS
8,BINOUDIERE (GAEC DE LA),2101,60.0,u,50,BOVINS
9,BINOUDIERE (GAEC DE LA),2101,80.0,u,50,BOVINS
...,...,...,...,...,...,...
23680,DE LA CLINERIE (GAEC) LEFEVRE Christian,2102,1059.0,Animaux-Eq,45,VOLAILLES
23682,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,Animaux-Eq,45,VOLAILLES
23684,DES JOURNETS (SCEA),2102,1857.0,Animaux-Eq,41,PORCS
23685,EARL DES MUSSETS,2111,40000.0,u,41,VOLAILLES


convert u to Animaux-Eq

In [37]:
def u_to_animaux_eq(row):
    if row.unite == 'Animaux-Eq': # skip treatment
        return row

    if row.type == "BOVINS":
        row.quantite *= 1.0
        row.unite = "Animaux-Eq"

    if row.type == "PORCS":
        row.quantite *= 0.14
        row.unite = "Animaux-Eq"

    if row.type == "VOLAILLES":
        row.quantite *= 0.01
        row.unite = "Animaux-Eq"
    
    return row

farms_stocks = farms_stocks.apply(u_to_animaux_eq, axis=1)

In [38]:
farms_stocks = pd.DataFrame({'quantite_mean' : farms_stocks.groupby(
    ["name", "rubric_id", "unite", "departement", "type"])['quantite'].mean()}).reset_index()

In [39]:
farms_stocks

,name,rubric_id,unite,departement,type,quantite_mean
0,2 L'oiselière (GAEC),2101,Animaux-Eq,50,BOVINS,130.000000
1,33 RUE DU BOCAGE (GAEC),2102,Animaux-Eq,50,PORCS,747.000000
2,4 CANTONS DUBRON (GAEC DES ),2101,Animaux-Eq,62,BOVINS,215.000000
3,4 CANTONS DUBRON (GAEC DES ),2102,Animaux-Eq,62,BOVINS,384.000000
4,4 CANTONS DUBRON (GAEC DES ),2111,Animaux-Eq,62,BOVINS,22500.000000
...,...,...,...,...,...,...
10842,earl la roche marteau,2101,Animaux-Eq,36,BOVINS,180.000000
10843,gene +,2102,Animaux-Eq,62,PORCS,1478.200000
10844,quatres sites (GAEC des),2101,Animaux-Eq,19,BOVINS,107.666667
10845,quatres sites (GAEC des),2102,Animaux-Eq,19,BOVINS,830.000000


In [41]:
farms_stocks["departement_unite_rank"] = farms_stocks.groupby(
    ["departement"])['quantite_mean'].rank(ascending=False)

In [42]:
farms_stocks["france_unite_rank"] = farms_stocks.groupby(
    ["unite"]
)["quantite_mean"].rank(ascending=False)

In [43]:
farms_stocks.sort_values(by=["france_unite_rank"], inplace=True)

In [44]:
farms_stocks

,name,rubric_id,unite,departement,type,quantite_mean,departement_unite_rank,france_unite_rank
7380,GUITTON (EARL),3660,Animaux-Eq,49,BOVINS,319500.0,1.0,1.0
8684,PLU ALEXANDRE,3660,Animaux-Eq,72,BOVINS,290465.0,1.0,2.0
4539,GAEC CHABAUTY,3660,Animaux-Eq,79,BOVINS,260000.0,1.0,3.0
780,DASCI Zafer,3660,Animaux-Eq,56,BOVINS,240000.0,1.0,4.0
6742,GAEC LUMINEAU CHARRIER,3660,Animaux-Eq,85,BOVINS,234240.0,1.0,5.0
...,...,...,...,...,...,...,...,...
2396,EARL DU GRAND BOIS,2102,Animaux-Eq,59,PORCS,0.0,282.0,10844.5
10648,THIBAUT Gilbert,2111,Animaux-Eq,90,VOLAILLES,0.0,5.0,10844.5
8977,SAEM ABATTOIR DU PAYS DE CHARLIEU,2101,Animaux-Eq,42,BOVINS,0.0,75.0,10844.5
8272,MICHEL ERIC,2102,Animaux-Eq,22,PORCS,0.0,1695.0,10844.5


In [45]:
farms_stocks_best = farms_stocks[farms_stocks["departement_unite_rank"] == 1]

In [46]:
farms_stocks_best

,name,rubric_id,unite,departement,type,quantite_mean,departement_unite_rank,france_unite_rank
7380,GUITTON (EARL),3660,Animaux-Eq,49,BOVINS,319500.0,1.0,1.0
8684,PLU ALEXANDRE,3660,Animaux-Eq,72,BOVINS,290465.0,1.0,2.0
4539,GAEC CHABAUTY,3660,Animaux-Eq,79,BOVINS,260000.0,1.0,3.0
780,DASCI Zafer,3660,Animaux-Eq,56,BOVINS,240000.0,1.0,4.0
6742,GAEC LUMINEAU CHARRIER,3660,Animaux-Eq,85,BOVINS,234240.0,1.0,5.0
...,...,...,...,...,...,...,...,...
10505,SEPAB (SARL),2101,Animaux-Eq,34,BOVINS,1100.0,1.0,4282.5
10634,TERRAZZONI Frères (SCEA),2111,Animaux-Eq,20,VOLAILLES,900.0,1.0,5044.5
4156,ETABLISSEMENTS GUY HARANG,2102,Animaux-Eq,78,PORCS,800.0,1.0,5459.0
8008,LES GRILLONS (SARL),2111,Animaux-Eq,84,VOLAILLES,350.0,1.0,8149.0


## Export to json

In [47]:
farms_stocks_best.to_json("4.json", orient="records")